# predict-lack-of-url
skelley@air.org<br> 
sarora@air.org<br>
February 2019<br>

## Description
This notebook tries to explain bias in missing observations, which are mostly caused to a lack of an identified URL or if a URL exists, lack of successfully scraped website data.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import re

In [2]:
def clean_firm_name (firm):
    firm_clnd = re.sub('(\.|,| corporation| incorporated| llc| inc| international| gmbh| ltd)', '', firm, flags=re.IGNORECASE).rstrip()
    return firm_clnd

In [3]:
all_firms_data = pd.read_csv('../../data/orgs/all_firms.csv', header = None)
all_firms_data.columns = ['Company', 'url']
print (all_firms_data.shape)
all_firms_data.head()

(1492, 2)


,Company,url
0,Two Blades Foundation,2blades.org/
1,3M Innovative Properties Company,3m.com/
2,Advanced Aqua Group,aadvancedaqua.com/
3,ABB AB,abb.com/
4,AbbVie Inc.,abbvie.com/


In [4]:
firm_data = pd.read_csv('../../data/patents/measures/firm_level_patent_measures.csv')
firm_data['firm_clnd'] = firm_data['organization_clnd'].apply(clean_firm_name)
firm_data.head()

,organization_clnd,mean_citations_3,first_year,num_patents_all,num_patents_3,mean_assignees_all,mean_assignees_3,mean_inventors_all,mean_inventors_3,firm_clnd
0,22nd Century Limited,0.000000,2014,21,2,1.000000,1.000000,1.952381,2.000000,22nd Century Limited
1,3M Innovative Properties Company,1.666667,1999,10263,58,1.009744,1.017544,3.465458,4.754386,3M Innovative Properties Company
2,AAC Technologies Pte,1.000000,2012,206,1,1.038835,1.000000,1.951456,1.000000,AAC Technologies Pte
3,Aadigen,3.000000,2016,5,1,1.000000,1.000000,3.400000,4.000000,Aadigen
4,ABB AB,0.000000,2000,380,1,1.005263,2.000000,2.781579,4.000000,ABB AB


In [9]:
all_emp_data = pd.read_csv('../../data/orgs/emps/eager_emps_v3.csv')
all_emp_data['url'] = 1
all_emp_data.loc[pd.isna(all_emp_data['hit_url']), ['url']] = 0
print (all_emp_data.columns)
print ('Firms with missing urls:', len(all_emp_data.loc[all_emp_data['url'] == 0]))
all_emp_data = all_emp_data[['firm', 'lookup_firm', 'firm_length', 'url', 'public', 'acquired_merged', 'nano', 'green', 'synbio', 'max_emps']]
print (all_emp_data.shape)
all_emp_data.head()

# unlabeled.loc[np.isnan(unlabeled['share_of_sentences_ftr']), 'share_of_sentences_ftr'] = 0 

Index(['firm', 'lookup_firm', 'firm_length', 'name_clnd', 'name_length',
       'manual_fix', 'hit_url', 'hit_url_length', 'rank', 'matches', 'public',
       'acquired_merged', 'Votes', 'Group', 'nano', 'green', 'synbio',
       'count_all_ind', 'max_emps', 'size_state', 'url'],
      dtype='object')
Firms with missing urls: 199
(1487, 10)


,firm,lookup_firm,firm_length,url,public,acquired_merged,nano,green,synbio,max_emps
0,Integrated Solar Technology,Integrated Solar Technology,27.0,1,0,0,0,1,0,71
1,Graphene Technologies,Graphene Technologies,21.0,1,0,0,1,0,0,79
2,Proton Power,Proton Power,12.0,1,0,0,0,1,0,74
3,Renewable Algal Energy,Renewable Algal Energy,22.0,1,0,0,0,1,0,90
4,Metabolix,Metabolix,9.0,1,0,0,1,0,0,66


In [6]:
len(all_emp_data)

1487

In [11]:
#about 50% merge, which we can use for the preliminary analysis
data = pd.merge(firm_data, all_emp_data, left_on = 'firm_clnd', right_on = 'lookup_firm', how = 'outer', indicator = True)
data._merge.value_counts()

both          1314
left_only      229
right_only     173
Name: _merge, dtype: int64

In [12]:
len(data[data._merge != 'right_only'][data['url'] == 1])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


1143

In [13]:
len(data[data._merge == 'both'][data['url'] == 1])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


1143

In [14]:
len(data[data._merge != 'right_only'][data['url'] == 0])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


171

In [85]:
full_data = data[data._merge == 'both']
#we do have some both with and without urls
full_data['url'].value_counts()

1.0    1143
0.0     171
Name: url, dtype: int64

In [86]:
print (full_data.columns)
full_data['log_num_patents_all'] = full_data['num_patents_all'].apply(np.log)
full_data['log_num_patents_3_all'] = full_data['num_patents_3'].apply(np.log)

Index(['organization_clnd', 'mean_citations_3', 'first_year',
       'num_patents_all', 'num_patents_3', 'mean_assignees_all',
       'mean_assignees_3', 'mean_inventors_all', 'mean_inventors_3',
       'firm_clnd', 'firm', 'lookup_firm', 'firm_length', 'url', 'public',
       'acquired_merged', 'nano', 'green', 'synbio', 'max_emps', '_merge'],
      dtype='object')


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [87]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from statsmodels.formula.api import logit

In [88]:
x_cols = [col for col in full_data.columns if not col in ['_merge', 'firm', 'lookup_firm', 'firm_clnd', 'organization_clnd', 'url', 'acquired_merged']]
xs = full_data[x_cols].fillna(0)
y = full_data['url']

xs.head()

,mean_citations_3,first_year,num_patents_all,num_patents_3,mean_assignees_all,mean_assignees_3,mean_inventors_all,mean_inventors_3,firm_length,public,nano,green,synbio,max_emps,log_num_patents_all,log_num_patents_3_all
0,0.000000,2014.0,21.0,2.0,1.000000,1.000000,1.952381,2.000000,20.0,0.0,0.0,0.0,1.0,734.0,3.044522,0.693147
1,1.666667,1999.0,10263.0,58.0,1.009744,1.017544,3.465458,4.754386,32.0,0.0,1.0,1.0,1.0,60593.0,9.236300,4.060443
2,1.000000,2012.0,206.0,1.0,1.038835,1.000000,1.951456,1.000000,20.0,0.0,1.0,0.0,0.0,2174.0,5.327876,0.000000
3,3.000000,2016.0,5.0,1.0,1.000000,1.000000,3.400000,4.000000,7.0,1.0,1.0,0.0,0.0,0.0,1.609438,0.000000
4,0.000000,2000.0,380.0,1.0,1.005263,2.000000,2.781579,4.000000,6.0,0.0,1.0,0.0,0.0,100487.0,5.940171,0.000000


## Logit

In [89]:
logit_m = logit("url ~ first_year + log_num_patents_all" 
                   "+ mean_assignees_all + mean_inventors_all + firm_length + "
                   " + nano + green + synbio + max_emps", full_data).fit()

Optimization terminated successfully.
         Current function value: 0.348306
         Iterations 11


In [90]:
print(logit_m.summary())

                           Logit Regression Results                           
Dep. Variable:                    url   No. Observations:                 1313
Model:                          Logit   Df Residuals:                     1303
Method:                           MLE   Df Model:                            9
Date:                Wed, 27 Feb 2019   Pseudo R-squ.:                 0.09622
Time:                        23:09:21   Log-Likelihood:                -457.33
converged:                       True   LL-Null:                       -506.01
                                        LLR p-value:                 5.342e-17
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              36.3475     18.981      1.915      0.056      -0.855      73.550
first_year             -0.0172      0.009     -1.829      0.067      -0.036       0.001
log_num_patents_

## Decision tree

In [30]:
clf = DecisionTreeClassifier()
grid = GridSearchCV(clf, {'max_depth' : [2, 3, 4, 5,6,10]}, scoring='f1')
grid.fit(xs, y)
grid.best_params_

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'max_depth': 3}

In [31]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

In [34]:
clf = DecisionTreeClassifier(random_state =42, max_depth=2)
clf.fit(xs, y)
precision = np.mean(cross_val_score(clf, xs, y, scoring = 'precision', cv=cv))
recall = np.mean(cross_val_score(clf, xs, y, scoring = 'recall', cv=cv))
f1 = np.mean(cross_val_score(clf, xs, y, scoring = 'f1', cv=cv))

In [35]:
y.value_counts()

1.0    1143
0.0     171
Name: url, dtype: int64

In [36]:
recall

0.9889810940896864

In [37]:
f1

0.9301144837929453

In [38]:
feature_imp = sorted(list(zip(x_cols, clf.feature_importances_)) , key=lambda x: x[1])
feature_imp

[('mean_citations_3', 0.0),
 ('first_year', 0.0),
 ('num_patents_all', 0.0),
 ('num_patents_3', 0.0),
 ('mean_assignees_all', 0.0),
 ('mean_assignees_3', 0.0),
 ('mean_inventors_all', 0.0),
 ('mean_inventors_3', 0.0),
 ('public', 0.0),
 ('nano', 0.0),
 ('green', 0.0),
 ('synbio', 0.0),
 ('firm_length', 0.12618514273450518),
 ('max_emps', 0.8738148572654949)]

In [39]:
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold


# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %s else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()

The binary tree structure has 7 nodes and has the following tree structure:
node=0 test node: go to node 1 if X[:, 13] <= 5.5 else to node 4.
	node=1 test node: go to node 2 if X[:, 8] <= 31.5 else to node 3.
		node=2 leaf node.
		node=3 leaf node.
	node=4 test node: go to node 5 if X[:, 13] <= 408.0 else to node 6.
		node=5 leaf node.
		node=6 leaf node.



In [40]:
#sanity check
# Sarah -- where does preds come from? 
relevant_xs = xs[['firm_length', 'max_emps']]
relevant_xs['preds'] = preds

NameError: name 'preds' is not defined

In [102]:
relevant_xs[relevant_xs['max_emps'] < 2.5][relevant_xs['firm_length']<22.5].head()

/Users/skelley/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,firm_length,max_emps,preds
3,7.0,0.0,1.0
12,7.0,2.0,1.0
28,13.0,0.0,1.0
44,7.0,0.0,1.0
62,22.0,0.0,1.0


In [104]:
relevant_xs[relevant_xs['max_emps'] < 2.5][relevant_xs['firm_length']>22.5].head()

/Users/skelley/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,firm_length,max_emps,preds
34,31.0,2.0,0.0
37,28.0,2.0,0.0
173,40.0,0.0,0.0
180,24.0,0.0,0.0
183,38.0,0.0,0.0


In [106]:
relevant_xs[2.5 < relevant_xs['max_emps']][relevant_xs['max_emps'] < 791.5].head()

/Users/skelley/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,firm_length,max_emps,preds
0,20.0,734.0,1.0
6,29.0,706.0,1.0
11,14.0,365.0,1.0
13,18.0,204.0,1.0
14,21.0,11.0,1.0


In [108]:
relevant_xs[relevant_xs['max_emps'] > 2.5][relevant_xs['max_emps']>791.5].head()

/Users/skelley/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,firm_length,max_emps,preds
1,32.0,60593.0,1.0
2,20.0,2141.0,1.0
4,6.0,100487.0,1.0
5,12.0,134800.0,1.0
7,16.0,99000.0,1.0
